### This Notebook controls the arima model for our analysis.

**The model is stored in the "arima_model.py" file, but it runs from this notebook.**

In [ ]:
import importlib
import pandas as pd
import arima_model as arm
import data_getter as data_con

importlib.reload(arm) # Simply reloads the module in case of changes.

In [ ]:
all_df = data_con.run_queries()
master_table = data_con.join_tables(all_df)

In [ ]:
prediction_dataframes = []
age_ranges = master_table.AgeGroup.unique().tolist()
age_ranges = [_ for _ in age_ranges if str(_) != 'nan']
all_fips = master_table.FIPS.unique().tolist()
total = len(all_fips) * len(age_ranges)

print('\nApproximate Run Time: 90 minutes\n')
cntr = 0
for i, age_range in enumerate(age_ranges):
    for ii, target_fips in enumerate(all_fips):
        params = [target_fips, age_range]
        target = 'MedianIncome'
        try:
            # Send data to the model
            graph_ready, export_ready = arm.dispatcher(master_table, target, params)
        except Exception as E:
            print(f'Failure: {E}, params: {params}')
            cntr += 1
            continue
        export_ready['AgeGroup'] = age_range
        prediction_dataframes.append(export_ready)
        cntr += 1
        print(f'{cntr} of {total} completed')

In [ ]:
main_frame = pd.DataFrame()
for frame in prediction_dataframes:
    frame['FIPS'] = frame['FIPS'].tolist()[0]
    main_frame = pd.concat([main_frame, frame])

main_frame.to_csv('PredictedIncomeAfterFinalRun.csv',index = False)